## Import dependencies

In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import neurokit2 as nk
import matplotlib.pyplot as plt
from bids.layout import BIDSLayout
from peakdet import Physio, operations, save_physio, load_physio

## Instantiate path

Since our data are organized in BIDS format, we can use `pybids` querying functionalities.

---

To learn more about [pybids](https://bids-standard.github.io/pybids/).
<br>To learn more about the BIDS format, check out the [BIDS paper](https://doi.org/10.1038/sdata.2016.44) and the [BIDS spec](https://bids-specification.readthedocs.io/en/stable/).

In [ ]:
path = '/path/to/the/dataset'
layout = BIDSLayout(path, validate=False, is_derivative=True)

The BIDSLayout object can be used to return subject id that are found in `path`, or session and run id for a specific subject. Let's see how it works.

In [ ]:
subjects = layout.get_subject()
# Let's see the subject id that we have in our directory
print(subjects)

In [ ]:
# To get the session id...
sessions = layout.get_session()
print(sessions)

Here we get the session id across all participants. However, the number of sessions and session ids could change from one participant to another. If we want to check the sessions related to one specific subject, we can use the `subject` parameter.

In [ ]:
sessions_06 = layout.get_session(subject='06')
print(sessions_06)

We can do the same for the runs. To get either the run ids across all subjects, or for a specific subject and a specific session.

In [ ]:
# Across participants and sessions
runs = layout.get_run()
print(runs)

# Within a specific participant across sessions
runs_06 = layout.get_run(subject='06')
print(runs_06)

# Within a specific participant for a specific session
runs_06_001 = layout.get_run(subjest='06', session='001')
print(runs_06_001)

## Load data

We are going to load two different files:

- The first file (content store in `preproc_physio`) contains the preprocessed timeseries (continuous data). 

- The second file (content store in `events_physio`) contains features that were extracted from the preprocessed timeseries (sparse data).

The features are the following:
- **PPG**: systolic peak (var: `systolic_peak_corrected`)
- **ECG**: r peak (var: `r_peak_corrected`)
- **RSP**: index at the maximum inhalation amplitude (var: `inhale_max`); index at the maximum exhalation amplitude (var: `exhale_max`)
- **EDA**: indext at the maximum amplitude of the skin conductance response (var: `scr_peak`); onset of the skin conductance response (var: `scr_onset`)

In [2]:
# Defining the feature name related to each modality
feat_dict = {
    'PPG': {
        'feat': 'systolic_peak_corrected',
        'feat_corrected': 'systolic_peak_manually_corrected'
    },
    'ECG': {
        'feat': 'r_peak_corrected',
        'feat_corrected': 'r_peak_manually_corrected'
    },
    'RSP': {
        'feat': 'inhale_max',
        'feat_corrected': 'inhale_max_manually_corrected',
        'feat_trough': 'exhale_max',
        'feat_trough_corrected': 'exhale_max_manually_corrected'
    },
    'EDA': {
        'feat': 'scr_peak',
        'feat_corrected': 'scr_peak_manually_corrected',
        'feat_trough': 'scr_onset',
        'feat_trough_corrected': 'scr_onset_manually_corrected'
    }
}

In [ ]:
def load_preproc_data(layout, sub, ses, run, modality="EDA", sampling_rate=1000):
    """
    Parameters
    ----------
    sub: str
        subject id (e.g., '01').
    ses: str
        session id (e.g., '001').
    run: str
        run id (e.g, '01').
    modality: str
        physiological modality. Could either be "ECG", "PPG", "RSP" or "EDA".
        Default to "EDA".
    
    Returns
    -------
    physio: Physio
        Physio object containing the preprocessed timeseries and the extracted peaks (and troughs).
    """
    
    # Load physio events to retrieve the peaks and troughs
    events_physio= layout.get(subject=sub, session=ses, run=run, suffix='events')
    entities = events_physio[0].get_entities()
    events_physio = pd.read_csv(events_physio[0], sep='\t')
    
    # Load preprocessed timeseries
    preproc_physio = layout.get(subject=sub, session=ses, run=run, suffix='physio', extension='tsv.gz')
    preproc_physio = pd.read_csv(preproc_physio[0], sep='\t')
    
    # Create Physio object
    physio = Physio(np.array(preproc_physio[f'{modality}_clean']), fs=sampling_rate)
    physio = operations.peakfind_physio(physio)
    
    # Retrieve features name based on `modality`
    feat = feat_dict[modality]['feat']
    feat_corrected = feat_dict[modality]['feat_corrected']
    
    physio._metadata['peaks'] = np.array(events_physio[events_physio['trial_type']==feat]['onset']*sampling_rate).astype(int)
    
    if modality in ["EDA", "RSP"]:
        feat_trough =  feat_dict[modality]['feat_trough']
        feat_trough_corrected = feat_dict[modality]['feat_trough_corrected']
        
        physio._metadata['troughs'] = np.array(events_physio[events_physio['trial_type']==feat_trough]['onset']*sampling_rate).astype(int)
    
    return physio

In [ ]:
sub = '01'
ses = '001'
run = '01'
modality = 'EDA'
sampling_rate = 1000

In [ ]:
physio = load_preproc_data(layout, sub, ses, run, modality=modality, sampling_rate=sampling_rate)

## Manual correction

For the manual QC, we will use [peakdet](https://peakdet.readthedocs.io/en/latest/index.html), more specifically the [`edit_physio` function](https://peakdet.readthedocs.io/en/latest/user_guide/editing.html).

In [ ]:
%matplotlib qt
physio = operations.edit_physio(physio)

In [ ]:
# The history shows the index related to added/deleted peaks
print(physio.history)